# Updating data with most recent matches

In [17]:
import pandas as pd
import numpy as np
import requests
import json
import time

In [18]:
# Read in the most recent dataframe

matches = pd.read_pickle('/Users/christophermarker/Documents/Pipeline_sample/assets/master_match_list')

In [19]:
# Assign a value to the most recent match in my current database

most_recent = list(matches.match_id)[0]

In [20]:
# Call the API to get the most recent 100 professional matches

response = requests.get('https://api.opendota.com/api/proMatches')
updates = pd.DataFrame.from_dict(response.json(), orient='columns')

In [21]:
# Create a value to remain constant until stopping condition is found

n = 1

In [22]:
# Check to see if my most recent match was contained within the most recent 100 professional matches
# If the match was not found, continue calling the API until the most recent professional match is found

for i in response.json():
    if i['match_id'] != most_recent:
        continue
    else:
        n = 0

while n == 1:          
    response = requests.get('https://api.opendota.com/api/proMatches?less_than_match_id=' + 
                            str(list(updates.match_id)[-1]))
    if response.status_code == 200:
        df = pd.DataFrame.from_dict(response.json(), orient='columns')
        updates = updates.append(df)
    else:
        time.sleep(10.0)
        pass
    
    try:
        for i in response.json():
            if i['match_id'] != most_recent:
                continue
            else:
                n = 0
    except:
        pass

In [23]:
# Combine the updates with the main database

updated_matches = updates.append(matches)

In [24]:
# Sort values to put newest matches at the top of the dataframe
# Remove matches with duplicate IDs

updated_matches.sort_values(by=['match_id'], axis=0, ascending=False, inplace=True)
updated_matches.drop_duplicates(subset='match_id', keep='first', inplace=True)

In [25]:
# A print statement to tell the user how many professional matches have been recorded since the last update

print('There have been ' + str(updated_matches.shape[0] - matches.shape[0]) + 
      ' professional matches played since the last update.')

There have been 95 professional matches played since the last update.


In [26]:
# Ressetting the index of the dataframe

updated_matches.reset_index(inplace = True)
updated_matches.drop('index', axis=1, inplace = True)

In [27]:
# Updating the master match list

updated_matches.to_pickle('/Users/christophermarker/Documents/Pipeline_sample/assets/master_match_list')

In [28]:
# Creating a timestamped backup version of the new master match list

updated_matches.to_pickle('/Users/christophermarker/Documents/Pipeline_sample/assets/' + str(int(time.time())) + '_' +
                  'previous_match_list')